<div class="alert alert-info">

## Introduction


</div>

In [45]:
#import
import os

import numpy as np
import pandas as pd
from hashlib import sha1

from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import accuracy

<div class="alert alert-info">

## Data Description<a name="2"></a>
Given the large size of the dataset, only 10000 rows of the dataset is used for the models.
This project utilizes a comprehensive dataset sourced from Kaggle, which can be accessed via the following link: (https://www.kaggle.com/datasets/beaglelee/amazon-reviews-us-books-v1-02-tsv-zip). The dataset consists of 15 columns and encompasses a substantial total of 3,105,370 rows, providing rich insights into customer feedback and product ratings specifically within the book category.

Due to the extensive size of the dataset, a subset of 10,000 rows has been selected for analysis and modeling. This reduction allows for efficient processing while still capturing the diverse range of reviews and ratings present in the original dataset.
</div>

In [21]:
# Data
data = pd.read_csv("data/amazon_reviews_us_Books_v1_02.tsv", sep='\t', on_bad_lines='skip')


In [22]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12076615,RQ58W7SMO911M,0385730586,122662979,Sisterhood of the Traveling Pants (Book 1),Books,4.0,2.0,3.0,N,N,this book was a great learning novel!,this boook was a great one that you could lear...,2005-10-14
1,US,12703090,RF6IUKMGL8SF,0811828964,56191234,The Bad Girl's Guide to Getting What You Want,Books,3.0,5.0,5.0,N,N,Fun Fluff,If you are looking for something to stimulate ...,2005-10-14
2,US,12257412,R1DOSHH6AI622S,1844161560,253182049,"Eisenhorn (A Warhammer 40,000 Omnibus)",Books,4.0,1.0,22.0,N,N,this isn't a review,never read it-a young relative idicated he lik...,2005-10-14
3,US,50732546,RATOTLA3OF70O,0373836635,348672532,Colby Conspiracy (Colby Agency),Books,5.0,2.0,2.0,N,N,fine author on her A-game,Though she is honored to be Chicago Woman of t...,2005-10-14
4,US,51964897,R1TNWRKIVHVYOV,0262181533,598678717,The Psychology of Proof: Deductive Reasoning i...,Books,4.0,0.0,2.0,N,N,Execellent cursor examination,Review based on a cursory examination by Unive...,2005-10-14


<div class="alert alert-info">

## Exploratory Data Analysis(EDA) <a name="3"></a>

This section outlines the exploratory data analysis techniques employed to extract meaningful insights from the dataset, guiding the subsequent steps in model development.

To create a focused subset for analysis, we identified the 1,000 unique product IDs with the highest number of reviews. From this selection, we sampled a total of 10,000 rows corresponding to their reviews. As a result, our final subset comprises 10,000 rows, encompassing 993 distinct products and 9,993 associated reviews. This approach ensures a diverse representation of products while allowing us to conduct a thorough examination of customer feedback.

</div>

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105370 entries, 0 to 3105369
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   marketplace        object 
 1   customer_id        int64  
 2   review_id          object 
 3   product_id         object 
 4   product_parent     int64  
 5   product_title      object 
 6   product_category   object 
 7   star_rating        float64
 8   helpful_votes      float64
 9   total_votes        float64
 10  vine               object 
 11  verified_purchase  object 
 12  review_headline    object 
 13  review_body        object 
 14  review_date        object 
dtypes: float64(3), int64(2), object(10)
memory usage: 355.4+ MB


In [24]:
data.isnull().sum()

marketplace            0
customer_id            0
review_id              0
product_id             0
product_parent         0
product_title          0
product_category       0
star_rating            4
helpful_votes          4
total_votes            4
vine                   4
verified_purchase      4
review_headline       57
review_body            4
review_date          133
dtype: int64

In [25]:
data = data.dropna()

In [26]:
data.replace(['null', 'N/A', '', ' '], np.nan, inplace=True)

In [27]:
data.isnull().sum()

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
dtype: int64

In [28]:
data.nunique()

marketplace                1
customer_id          1502265
review_id            3105184
product_id            779692
product_parent        666003
product_title         713665
product_category           1
star_rating                5
helpful_votes            942
total_votes             1024
vine                       2
verified_purchase          2
review_headline      2456998
review_body          3070458
review_date             3575
dtype: int64

In [30]:
# Getting subset
# 1. Group by product_id and count the number of reviews for each product
product_review_counts = data.groupby('product_id').size().reset_index(name='review_count')

# 2. Sort by review count in descending order and get the top 1000 products
top_1000_products = product_review_counts.sort_values(by='review_count', ascending=False).head(1000)

# 3. Filter the original dataset to get reviews only for the top 1000 products
top_1000_product_reviews = data[data['product_id'].isin(top_1000_products['product_id'])]

# 4. Sample 10000 rows from the filtered dataset
subset = top_1000_product_reviews.sample(n=10000, random_state=42) 

# Display the resulting subset
subset.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
2141488,US,51218382,R2P7ER0N02OH1B,0553106635,549456361,"A Storm of Swords (A Song of Ice and Fire, Boo...",Books,5.0,1.0,3.0,N,N,I'm not even done yet!,I started reading some of the reviews already ...,2001-01-13
875012,US,39315466,RPBOZVFMMGFTL,0345325818,619220705,The Silmarillion,Books,5.0,17.0,17.0,N,N,The Silmarillion for the newer Tolkien fans.,One of the things that must be understood abou...,2003-12-06
2200658,US,52930440,R2X51YQRJ65FNF,0679879269,731430286,"The Amber Spyglass (His Dark Materials, Book 3)",Books,1.0,24.0,49.0,N,N,Disappointing and dishonest,After a wonderful beginning in The Golden Comp...,2000-11-23
818842,US,35305931,R3OIYBWPGIUUR3,0761104844,280019609,"1,000 Places to See Before You Die: A Traveler...",Books,5.0,9.0,10.0,N,N,Excellent Book!,This book is awesome for anyone who travels. ...,2004-01-20
1821732,US,52153269,R3B9N5GYR7ZBPF,0786866020,257333183,Fish: A Proven Way to Boost Morale and Improve...,Books,5.0,4.0,6.0,N,N,Move over &quot;cheese&quot; it is time for Fi...,An interesting effective yet light read on how...,2001-10-02


In [31]:
subset.nunique()

marketplace              1
customer_id           9633
review_id            10000
product_id             993
product_parent         880
product_title          933
product_category         1
star_rating              5
helpful_votes          160
total_votes            205
vine                     1
verified_purchase        2
review_headline       9439
review_body           9993
review_date           2644
dtype: int64

In [32]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 2141488 to 2464275
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   marketplace        10000 non-null  object 
 1   customer_id        10000 non-null  int64  
 2   review_id          10000 non-null  object 
 3   product_id         10000 non-null  object 
 4   product_parent     10000 non-null  int64  
 5   product_title      10000 non-null  object 
 6   product_category   10000 non-null  object 
 7   star_rating        10000 non-null  float64
 8   helpful_votes      10000 non-null  float64
 9   total_votes        10000 non-null  float64
 10  vine               10000 non-null  object 
 11  verified_purchase  10000 non-null  object 
 12  review_headline    10000 non-null  object 
 13  review_body        10000 non-null  object 
 14  review_date        10000 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 1.2+ MB


In [33]:
subset.to_csv('amazon_reviews_subset.csv', index=False)


<div class="alert alert-info">
    
## Collaborative Filtering
**Collaborative Filtering** is a widely-used technique for addressing the challenge of missing entries in a utility matrix, leveraging user behavior and interactions to make recommendations. This approach operates on the principle that users who have agreed in the past will continue to agree in the future, allowing the model to infer preferences based on the preferences of similar users.

This method can be likened to advanced dimensionality reduction techniques such as Latent Semantic Analysis (LSA) or Truncated Singular Value Decomposition (SVD). By capturing the underlying relationships between users and items, collaborative filtering helps to predict missing values, enhancing the accuracy and relevance of recommendations.

In this project, we will implement collaborative filtering as our baseline model to improve user experience by personalizing content based on historical data, thus enabling more informed decision-making.
</div>

In [38]:
coll_data = subset[['customer_id', 'product_id', 'star_rating']].reset_index(drop=True)
coll_data.head()

,customer_id,product_id,star_rating
0,51218382,0553106635,5.0
1,39315466,0345325818,5.0
2,52930440,0679879269,1.0
3,35305931,0761104844,5.0
4,52153269,0786866020,5.0


In [39]:
# data splitting
user_key = "customer_id"
item_key = "product_id"

X = coll_data.copy()
y = coll_data[user_key]

In [42]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=123)
X_train.shape, X_valid.shape

((8000, 3), (2000, 3))

In [43]:
# utility Matrix
train_mat = X_train.pivot_table(index='customer_id', columns='product_id', values='star_rating')
valid_mat = X_valid.pivot_table(index='customer_id', columns='product_id', values='star_rating')


In [48]:
# Using surprise package
reader = Reader()
data = Dataset.load_from_df(coll_data, reader)  

k = 10
algo = SVD(n_factors=k, random_state=42)

In [50]:
pd.DataFrame(cross_validate(algo, data, measures=["RMSE"], cv=5, verbose=True))

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3456  1.3169  1.3468  1.3605  1.3384  1.3416  0.0143  
Fit time          0.02    0.01    0.01    0.01    0.01    0.02    0.00    
Test time         0.01    0.00    0.00    0.00    0.00    0.00    0.00    


,test_rmse,fit_time,test_time
0,1.345564,0.022206,0.006118
1,1.316888,0.014180,0.003129
2,1.346834,0.013556,0.002931
3,1.360520,0.013715,0.002925
4,1.338389,0.013792,0.002928
